### Set up a client to handle authentication etc with geoserver

In [ ]:
import requests
import functools
from pprint import pprint
from IPython.core.display import display, HTML

BASE_URL = "http://127.0.0.1:8080/geoserver/rest"

class Client(object):
  def __init__(self, url, username='admin', password='geoserver'):
    self.base_url = url
    self.auth = {'auth': (username, password)}
    
  def _proxy(self, method, uri, *args, **kwargs):
    kwargs.update(self.auth)
    return method(self.base_url + uri, *args, **kwargs)

  def get(self, *args, **kwargs):
    return self._proxy(requests.get, *args, **kwargs)
  
  def put(self, *args, **kwargs):
    return self._proxy(requests.put, *args, **kwargs)
  
  def post(self, *args, **kwargs):
    return self._proxy(requests.post, *args, **kwargs)
  
  def delete(self, *args, **kwargs):
    return self._proxy(requests.delete, *args, **kwargs)
  
  def head(self, *args, **kwargs):
    return self._proxy(requests.head, *args, **kwargs)  
 
c = Client(BASE_URL)


### Make sure you have the geoserver VM running

this should show something like:

```
Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.
```

You can do this by going to the geoserver/ directory and running ```vagrant up```

In [ ]:
!cd ../geoserver && vagrant status

### Display geoserver status

This should ensure the client can successfully connect to your VM,  if you do not see the Geoserver 'Status' page then something is wrong!

In [ ]:
from IPython.core.display import display, HTML
display(HTML(c.get("/about/status").text))

### Create a workspace

In [ ]:
c.post("/workspaces.json", 
       json={'workspace': {'name': 'test'}}) 

In [ ]:
# Get the workspace
c.get("/workspaces/test.json").json()

### Get the rgb.tif file from data.kitware.com

In [ ]:
!wget -O /tmp/rgb.tif https://data.kitware.com/api/v1/file/57b471dd8d777f126827e6b7/download

### Create coveragestore with the coverage and generate layers 

In [ ]:
with open("/tmp/rgb.tif", 'rb') as fh:
  c.put("/workspaces/test/coveragestores/rgb/file.geotiff",
         params={"coverageName": 'rgb', 
                 "configure": "all"},
         headers={"Content-type": "image/tiff"},
         data=fh)

### Get the 'rgb' coverage

In [ ]:
current = c.get("/workspaces/test/coveragestores/rgb/coverages/rgb.json").json()
current

### Change some properties of the coverage

In [ ]:
current['coverage']['dimensions']['coverageDimension'][0]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][0]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][0]['range']['max'] = 255
current['coverage']['dimensions']['coverageDimension'][1]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][1]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][1]['range']['max'] = 255
current['coverage']['dimensions']['coverageDimension'][2]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][2]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][2]['range']['max'] = 255

c.put("/workspaces/test/coveragestores/rgb/coverages/rgb.json", json=current)

In [ ]:
# Get the layer
# c.get("/layers/rgb.json").json()

In [6]:
M.layers['osm_base'].config.get_vis_server()

In [7]:
s = M.layers['osm_base'].config.get_vis_server()

In [10]:
s.c.get("/workspaces/test.json").json()

{u'workspace': {u'coverageStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/coveragestores.json',
  u'dataStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/datastores.json',
  u'name': u'test',
  u'wmsStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/wmsstores.json'}}

In [ ]:
M.add_wms_layer("rgb", "http://127.0.0.1:8080/geoserver/ows")

In [ ]:
# List Layers
M.layers

# Select Layer
# M.layers['rgb']

# Get data from layer region - not yet implemented!
# M.layers['rgb'].region 

In [ ]:
M.remove_wms_layer('rgb')

In [ ]:
M.layers

In [ ]:
# Clean up,  delete the workspace
c.delete("/workspaces/test")